In [ ]:
# Idea: semantic search relevant FAQ and with that information use ChatGPT to generate a response



In [10]:
# Load jsonl
import json
import pandas as pd

def load_faq(faq_path:str = '../src/data/final/data_prepared_prepared.jsonl') -> list[dict]:
    with open(faq_path) as f:
        faq = [json.loads(line) for line in f]
    return faq

faq = load_faq()
pd.DataFrame(faq).head()

,prompt,completion
0,Что такое Техника Безопасности в нашей компании?,Техника безопасности - это набор правил и мер...
1,Какие основные риски и опасности на рабочем ме...,Основными рисками и опасностями на рабочем ме...
2,Какие виды личной защитной экипировки использу...,В нашей компании используется широкий спектр ...
3,Какие правила обращения с техническим оборудов...,Сотрудники должны следовать инструкциям по эк...
4,Как происходит оценка рисков на рабочем месте?,Оценка рисков на рабочем месте проводится спе...


In [11]:
# Get OPENAI_API_KEY from dotenv
from dotenv import load_dotenv
load_dotenv()
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# stopwords https://github.com/stopwords-iso/stopwords-ru/blob/master/stopwords-ru.txt


In [45]:
def load_stopwords(path:str='../src/data/final/tfidf_stopwords.txt') -> list[str]:
    res = []
    for l in open(path):
        res.append(l.strip())
    return res



def baseline_get_relevant_faq(query:str, faq_strings:list[str], top_n: int = 5) -> list:
    """Calculate cosine similarity between query and faq and return most relevant faq"""
    # Step 1: Extract keywords from query
    from sklearn.feature_extraction.text import TfidfVectorizer
    # https://www.machinelearningplus.com/nlp/cosine-similarity/
    # Create the Document Term Matrix
    stopwords_ru = load_stopwords()
    count_vectorizer = TfidfVectorizer(stop_words=stopwords_ru)

    # print(pd.DataFrame(faq_strings).head(10))
    # print(type(faq_strings))
    # print(type(faq_strings[0]))

    sparse_matrix = count_vectorizer.fit_transform(faq_strings)

    df = pd.DataFrame(sparse_matrix.toarray(), columns=count_vectorizer.get_feature_names_out())

    # Step 2: Calculate cosine similarity between query and faq
    from sklearn.metrics.pairwise import cosine_similarity
    query_vector = count_vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_vector, sparse_matrix).flatten()

    # Step 3: Return top_n most relevant faq
    related_docs_indices = cosine_similarities.argsort()[:-top_n - 1:-1]
    return [faq_strings[i] for i in related_docs_indices]

In [65]:
faq = load_faq()
faq_strings = [faq['prompt']+faq['completion'] for faq in faq]
relevant_faq = baseline_get_relevant_faq('Какие документы нужны для получения кредита?', faq_strings)

baseline_get_relevant_faq('Где лучшие пышки?', faq_strings)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  warnings.warn(


['Где лучшие пышки? В Политехе END',
 'Где лучшие пышки? СПбПУ им. Петра Великого. END',
 "Где лучшие пышки? СПбПУ им. Петра Великого. Кафе называется 'Пышки', по адресу: \u200bПолитехническая, 29лит 1 этаж Академическое, Калининский район, Санкт-Петербург, 194064 END",
 'Почему нашу компанию называют техногазелью? RuNetSoft называют техногазелью, потому что она является быстрорастущей инновационной, высокотехнологичной компанией, принадлежащей к сегменту малого и среднего бизнеса, и получила статус «газели» в рамках программы поддержки от АО «Корпорация «МСП».',
 'Какие меры следует принять для обеспечения безопасности в здании при возможности ЧС? Установите и поддерживайте систему пожарной безопасности, проверьте работоспособность и наличие необходимого оборудования, такого как огнетушители и дымовые извещатели. Проводите тренировочные учения по эвакуации сотрудников и знакомьте их с местом нахождения аварийных выходов. END']

In [63]:
# History is a list of dictionaries with keys 'role' and 'content'

def generate_answer(query: str, faq_strings: list[str], top_n: int = 5, history: list[dict] = None):
    # Step 1 get relevant FAQ if first
    related_faqs = baseline_get_relevant_faq(query, faq_strings, top_n=top_n)

    # Step 2 use ChatGPT to generate answer based on query and faq answer
    import openai
    openai.api_key = OPENAI_API_KEY

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content":
                f"""Привет, вот база данных faq: {related_faqs}\n"""
                f"""На их основе, ответь на следующий вопрос: {query}\n"""
                f"""Если имея эти факты нельзя ответить вопрос, попроси меня
                сформулировать вопрос иначе. \n""",
            },
        ],
        max_tokens=1000,
        stop=[" END"],
        temperature=0.69,
    )
    # Find the first response from the chatbot
    # that has text in it (some responses may not have text)
    for choice in response.choices:
        if "text" in choice:
            return choice.text

    answer = response.choices[0].message.content
    # If no response with text is found, return
    # the first response's content (which may be empty)
    # Step 3 return answer
    return answer

In [66]:
faq = load_faq()
faq_strings = [faq['prompt'] + faq['completion'] for faq in faq]
generate_answer('А как летать?', faq_strings, history=None)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  warnings.warn(


'Попроси меня сформулировать вопрос иначе, так как данная база данных не содержит информации, относящейся к данному запросу.'